## Experiments on _Rfam novel_ datasets

In [1]:
import numpy
import sklearn
import tensorflow
import matplotlib
import pandas

print(tensorflow.__version__)
print(sklearn.__version__)
print(numpy.__version__)
print(matplotlib.__version__)
print(pandas.__version__)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

1.13.1
0.21.3
1.18.5
3.1.1
0.24.2


In [2]:
import numpy as np
from tensorflow import keras
from sklearn import preprocessing

# Load labels 

train_labels=np.load("train_labels.npy")
val_labels=np.load("val_labels.npy")
test_labels=np.load("test_labels.npy")

num_classes = len(np.unique(train_labels))
print('Total classes: ',num_classes)

# Encodes labels to categorical

le = preprocessing.LabelEncoder()
le.fit(train_labels)
train_labels_num = le.transform(train_labels)
val_labels_num = le.transform(val_labels)
test_labels_num = le.transform(test_labels)

train_labels_bin = keras.utils.to_categorical(train_labels_num, num_classes)
val_labels_bin = keras.utils.to_categorical(val_labels_num, num_classes)
test_labels_bin = keras.utils.to_categorical(test_labels_num, num_classes)




Total classes:  88


In [ ]:
from utils.ExpConfiguration import *
from utils.modelUtils import *
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import f1_score
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)


padd = 'new' # CHANGE HERE to select other padding schemas (new, constant, random)

batch_size = 32
epochs = 20

# where results are stored
v_acc = {}

for nl in nlayers:
    venc = {}
    for en in seqEncoders:
        vbnoise = {}
        for bn in bnoise:
            print('n. layers=',nl,' Encoder=',en['filename'],' Noise=',str(bn), ' Padding=',padd)
            train_seq=np.load('train_' + en['filename'] + '_' + padd + '_' + str(bn) + '_seq.npy')
            val_seq=np.load('val_' + en['filename'] + '_' + padd + '_' + str(bn) + '_seq.npy')
            test_seq=np.load('test_' + en['filename'] + '_' + padd + '_' + str(bn) + '_seq.npy')

            if (nl>0):
                train_seq = keras.utils.to_categorical(train_seq)
                val_seq = keras.utils.to_categorical(val_seq)
                test_seq = keras.utils.to_categorical(test_seq)
            else:
                train_seq = train_seq.reshape((-1,np.prod(train_seq.shape[1:])))
                val_seq = val_seq.reshape((-1,np.prod(val_seq.shape[1:])))
                test_seq = test_seq.reshape((-1,np.prod(test_seq.shape[1:])))
                nmax = np.max(train_seq)
                train_seq = train_seq/nmax
                val_seq = val_seq/nmax
                test_seq = test_seq/nmax

            tf.keras.backend.clear_session()
            print(train_seq.shape)
            print(val_seq.shape)
            print(test_seq.shape)
            if (en['filename'] in ['1mer','2mer','3mer']):
                m=buildCNNModel(inshape=train_seq.shape[1:],num_classes=num_classes,nlayers=nl,cnndim=1)
            else:
                m=buildCNNModel(inshape=train_seq.shape[1:],num_classes=num_classes,nlayers=nl,cnndim=2)
            print(m.summary())
            m.compile(optimizer=keras.optimizers.Adam(lr=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
            m.fit(train_seq, train_labels_bin,verbose=1,
                  batch_size=batch_size,shuffle=True,
                  epochs=epochs,#validation_split=0.33,
                  validation_data=(val_seq, val_labels_bin))

            pred = m.predict(test_seq, verbose=1)
            predicted = np.argmax(pred, axis=1)
            
            vbnoise.update({str(bn) : predicted})
        venc.update({en['filename'] : vbnoise})
    v_acc.update({str(nl) : venc})




n. layers= 0  Encoder= 3mer  Noise= 0  Padding= new
(105864, 67)
(17324, 67)
(25342, 67)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 500)               34000     
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              501000    
_________________________________________________________________
dense_2 (Dense)              (None, 100)               100100    
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 88)                8888     

In [ ]:
import pickle

# save results on file
f = open('results/RfamNovel_' + padd + '.pckl', 'wb')
pickle.dump(v_acc, f)
f.close()

